In [1]:
import pandas as pd
import cv2
from keras.utils import img_to_array
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.backend import image_data_format
from keras.layers import Conv2D, Activation, BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense 
from keras.optimizers import Adam
import matplotlib.pyplot as plt

2023-04-12 16:56:54.847155: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
target=pd.read_csv('/Users/joshuaodugbemi/Downloads/Datasets/archive (4)/train.csv')

In [3]:
# 1 is for male and 0 is for female
target.head()

,image_names,class
0,11413.jpg,1
1,9863.jpg,1
2,997.jpg,0
3,5950.jpg,0
4,10454.jpg,1


In [4]:
target['images']='/Users/joshuaodugbemi/Downloads/Datasets/archive (4)/images/'+target['image_names']

In [5]:
target['images'][0]

'/Users/joshuaodugbemi/Downloads/Datasets/archive (4)/images/11413.jpg'

In [6]:
target.head()

,image_names,class,images
0,11413.jpg,1,/Users/joshuaodugbemi/Downloads/Datasets/archi...
1,9863.jpg,1,/Users/joshuaodugbemi/Downloads/Datasets/archi...
2,997.jpg,0,/Users/joshuaodugbemi/Downloads/Datasets/archi...
3,5950.jpg,0,/Users/joshuaodugbemi/Downloads/Datasets/archi...
4,10454.jpg,1,/Users/joshuaodugbemi/Downloads/Datasets/archi...


In [7]:
target=target.drop('image_names',axis=1)

In [8]:
target.head()

,class,images
0,1,/Users/joshuaodugbemi/Downloads/Datasets/archi...
1,1,/Users/joshuaodugbemi/Downloads/Datasets/archi...
2,0,/Users/joshuaodugbemi/Downloads/Datasets/archi...
3,0,/Users/joshuaodugbemi/Downloads/Datasets/archi...
4,1,/Users/joshuaodugbemi/Downloads/Datasets/archi...


In [9]:
data=[]
labels=[]

In [10]:
for img in target['images']:
    image=cv2.imread(img)
    image=cv2.resize(image,(96,96))
    image=img_to_array(image)
    data.append(image)

In [11]:
labels=target['class']

In [12]:
data=np.array(data,dtype='float')/255.0
labels=np.array(labels)

In [13]:
x_train,x_test,y_train,y_test=train_test_split(data,labels,test_size=0.3,random_state=42)

In [14]:
y_train=to_categorical(y_train,num_classes=2)
y_test=to_categorical(y_test,num_classes=2)

In [15]:
aug= ImageDataGenerator(rotation_range=25,width_shift_range=0.1,height_shift_range=0.1,
                        shear_range=0.2,zoom_range=0.2,horizontal_flip=True,fill_mode='nearest')

In [16]:
def build(width,height,depth,classes):
    model=Sequential()
    input_shape=(height,width,depth)
    chanDim=-1
    
    if image_data_format()=='channels_first':
        input_shape=(depth,height,width)
        chanDim=1
        
    model.add(Conv2D(32,(3,3),padding='same',input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64,(3,3),padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=chanDim))
    
    model.add(Conv2D(64,(3,3),padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128,(3,3),padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=chanDim))
    
    model.add(Conv2D(128,(3,3),padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(classes))
    model.add(Activation('sigmoid'))
    
    return model

In [17]:
model=build(96,96,3,2)

In [18]:
opt=Adam(learning_rate=1e-3, decay=1e-3/100)

In [19]:
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [20]:
final=model.fit_generator(aug.flow(x_train,y_train,batch_size=64), validation_data=(x_test,y_test),
                                   steps_per_epoch=len(x_train)//64, epochs=10, verbose=1)

Epoch 1/10


/var/folders/4v/14_p203d1_987b4lxcc7_b380000gp/T/ipykernel_2575/2877322802.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  final=model.fit_generator(aug.flow(x_train,y_train,batch_size=64), validation_data=(x_test,y_test),
2023-04-12 16:57:43.382168: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


133/133 [==============================] - 120s 889ms/step - loss: 0.7282 - accuracy: 0.6903 - val_loss: 0.7248 - val_accuracy: 0.5532
Epoch 2/10
133/133 [==============================] - 114s 859ms/step - loss: 0.5690 - accuracy: 0.7476 - val_loss: 0.8321 - val_accuracy: 0.5256
Epoch 3/10
133/133 [==============================] - 115s 866ms/step - loss: 0.4945 - accuracy: 0.7785 - val_loss: 0.6382 - val_accuracy: 0.6496
Epoch 4/10
133/133 [==============================] - 117s 880ms/step - loss: 0.4406 - accuracy: 0.8007 - val_loss: 0.3837 - val_accuracy: 0.8224
Epoch 5/10
133/133 [==============================] - 121s 908ms/step - loss: 0.3989 - accuracy: 0.8179 - val_loss: 0.3498 - val_accuracy: 0.8491
Epoch 6/10
133/133 [==============================] - 124s 933ms/step - loss: 0.3575 - accuracy: 0.8409 - val_loss: 0.2573 - val_accuracy: 0.8942
Epoch 7/10
133/133 [==============================] - 128s 959ms/step - loss: 0.3450 - accuracy: 0.8468 - val_loss: 0.2614 - val_accura

In [21]:
model.save('gender_classification.model')

2023-04-12 17:18:23.642569: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,32,32,32]
	 [[{{node inputs}}]]
2023-04-12 17:18:23.672682: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,16,16,64]
	 [[{{node inputs}}]]
2023-04-12 17:18:23.701646: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,8,8,128]
	 [[{{node inputs}}]]
20

INFO:tensorflow:Assets written to: gender_classification.model/assets


INFO:tensorflow:Assets written to: gender_classification.model/assets
